In [ ]:
# 5. Створення сезонних пар
print("🔄 Створення сезонних пар...")
pairs = []

for tile in ['T36UYA', 'T36UXA']:
    # Фільтруємо зображення для поточного тайлу
    tile_images = [img for img in folder_data if img['tile'] == tile]
    
    # Зображення для кожного сезону
    summer_images = [img for img in tile_images if img['season'] == 'summer']
    other_seasons = {
        'autumn': [img for img in tile_images if img['season'] == 'autumn'],
        'winter': [img for img in tile_images if img['season'] == 'winter'],
        'spring': [img for img in tile_images if img['season'] == 'spring']
    }
    
    # Обробка кожної вирубки
    for defor in tqdm(deforestation_by_tile[tile], desc=f"Обробка {tile}"):
        centroid = defor['geometry'].centroid
        
        # Створюємо пари для кожного сезону
        for season_name, season_images in other_seasons.items():
            for summer_img in summer_images:
                for other_img in season_images:
                    days_diff = abs((other_img['date'] - summer_img['date']).days)
                    
                    # Перевіряємо мінімальний часовий інтервал
                    if days_diff < MIN_DAYS_BETWEEN:
                        continue
                    
                    # Перевіряємо, чи вирубка видима в обох знімках
                    if not (Point(centroid).within(box(*get_tile_bounds(summer_img['tci_path']))) and
                            Point(centroid).within(box(*get_tile_bounds(other_img['tci_path'])))):
                        continue
                    
                    pairs.append({
                        'summer_img': summer_img,
                        'other_img': other_img,
                        'season_pair': f'summer_{season_name}',
                        'deforestation': defor,
                        'centroid': centroid,
                        'tile': tile
                    })

print(f"✅ Створено {len(pairs)} сезонних пар")